In [3]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import pandas as pd
from tqdm.notebook import tqdm
# import torch 
import json
from pprint import pprint
from concurrent.futures import ProcessPoolExecutor
import gc
import re
import string 
import numpy as np 
import os
import h5py

pd.set_option('use_inf_as_na', True)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)
# torch.__version__

/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_44282/3432076818.py:19: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('use_inf_as_na', True)


In [2]:
# !pip install pydrive
# !pip install --upgrade google-api-python-client oauth2client

# get clients_secret.json from google credentials in google console


# DOWNLOAD FILE
# file_id = 'your_file_id_here'  # Replace with your file ID
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('filename.txt')
# Upload a File:

# UPLOAD FILE
# upload_file = drive.CreateFile({'title': 'new_file.txt'})
# upload_file.SetContentFile('local_file.txt')
# upload_file.Upload()

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
gauth = GoogleAuth()
gauth.LoadClientConfigFile("datatraxer_gdrive_secret.json")
gauth.LocalWebserverAuth()  # Creates local webserver and automatically handles authentication.

drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=545847063411-p8bi7qrhiumhntq5sm9equtv1naohj4j.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


# download from gdrive

In [5]:
folder_id = '1kMfakNU9WtKVXaJ2X26nO8jdHthe_a0D'  # Replace with your folder ID
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
for file in file_list:
    # print(f"title: {file["title"]}, id: {file['id']}")
    try:
        downloaded = drive.CreateFile({'id': file['id']})
        downloaded.GetContentFile(f'data_raw/{file["title"]}')
        print(f'{file["title"]} - SUCCESS')
    except:
        print(f'{file["title"]} - FAIL')
        

March-2024.xlsx - SUCCESS
February-2-half-2024.xlsx - SUCCESS
February-1-half-2024.xlsx - SUCCESS
January-2-half-2024.xlsx - SUCCESS
January-1-half-2024.xlsx - SUCCESS
December-2023.xlsx - SUCCESS
November-2023.xlsx - SUCCESS
October-2023.xlsx - SUCCESS
September-2023.xlsx - SUCCESS
August-2023.xlsx - SUCCESS
July-2023.xlsx - SUCCESS


In [11]:
def filter_data(df, columns_to_load):
    print('before filter', df.shape)
    df = df[columns_to_load]
    df = df[df['type'] == 1]
    df = df.dropna(subset=['body'])
    df = df.sort_values(['userId', 'dateCreate'])
    return df

def clean_data(df):
    df['body'] = df['body'].str.strip()
    df['body'] = df['body'].str.replace('\t', '', regex=False)
    df = df.drop_duplicates(subset=['body', 'dateCreate'])
    df = df[df['body'].notna()]
    df['userId'] = df['userId'].astype(int)
    return df

def remove_consecutive_duplicates(df):
    df = df[df['body'] != df['body'].shift()]
    print('after filter', df.shape)
    return df

def remove_consecutive_persona_messages(df):
    df['remove'] = (df['sender'] == 'Persona') & (df['sender'].shift() == 'Persona')
    df['remove'] = df['remove'].fillna(False)
    df = df[~df['remove']]
    df.drop(columns=['remove'], inplace=True)
    print('after removing personas in a row', df.shape)
    return df

def remove_frequent_user_messages(df, topfreq=50):
    repeats = df[df['sender'] == 'User']['body'].value_counts()[:topfreq].index.values.tolist()
    df = df[~df['body'].isin(repeats)].copy()
    print('removed repeats', df.shape)
    return df

def concatenate_consecutive_user_messages(df):
    df['new_group'] = (df['sender'] != df['sender'].shift()) | (df['sender'] != 'User')
    df['group_id'] = df['new_group'].cumsum()
    df = df.groupby('group_id').agg({
        'sender': 'first',
        'body': '\n'.join,
        'userId': 'first',
        'dateCreate': 'first'
    }).reset_index(drop=True)
    return df

def clean_text(text):
    """
    Removes punctuation, symbols (excluding English and Russian), and extra spaces from text.
    """
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove symbols except English and Russian letters
    allowed_chars = string.ascii_letters + 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ '
    text = ''.join(c for c in text if c in allowed_chars)

    # Remove extra spaces
    text = ' '.join(text.split())

    return text

def create_paired_df(ori_df):
    # Shift the DataFrame to align questions with answers
    ori_df['next_sender'] = ori_df['sender'].shift(-1)
    ori_df['next_body'] = ori_df['body'].shift(-1)
    ori_df['next_dateCreate'] = ori_df['dateCreate'].shift(-1)

    # Filter to get only rows where the question is from the User and the next message is from Persona
    question_answer_pairs = ori_df[(ori_df['sender'] == 'User') & (ori_df['next_sender'] == 'Persona')]

    # Construct the resulting DataFrame or list of dictionaries
    result = question_answer_pairs.apply(lambda row: {
        'userId': row['userId'],
        'question': row['body'],
        'answer': row['next_body'],
        'dateCreate': row['dateCreate']
    }, axis=1).tolist()

    # Optional: Convert to a DataFrame
    paired_df = pd.DataFrame(result)
    return paired_df

# Example usage
columns_to_load = ['type', 'userId', 'dateCreate', 'body', 'sender']
files = [
 'March-2024.xlsx',
 'February-2-half-2024.xlsx',
 'February-1-half-2024.xlsx',
 'January-2-half-2024.xlsx',
 'January-1-half-2024.xlsx',
 'December-2023.xlsx',
 'November-2023.xlsx',
 'October-2023.xlsx',
 'September-2023.xlsx',
 'August-2023.xlsx',
 'July-2023.xlsx']
for file_name in files:
    try:
        ori_df = pd.read_excel(f'data_raw/{file_name}')
        ori_df = filter_data(ori_df, columns_to_load)
        ori_df = clean_data(ori_df)
        ori_df = remove_consecutive_duplicates(ori_df)
        ori_df = remove_consecutive_persona_messages(ori_df)
        ori_df = remove_frequent_user_messages(ori_df, topfreq=50)
        ori_df = concatenate_consecutive_user_messages(ori_df)
        ori_df['body'] = ori_df['body'].apply(clean_text)
        paired_df = create_paired_df(ori_df)
        paired_df.to_csv(f'data_paired/{file_name[:-5]}.csv')
    except Exception as e:
        print(f'Exception: {e}')

KeyboardInterrupt: 

In [8]:
ori_df[:5]

,sender,body,userId,dateCreate
0,Persona,ah there you are i was beginning to think youd forgotten about me winks im still here wearing that tight sheer dress and lacy lingerie i sent you a photo of earlier ive been waiting patiently for you to come back and take me in your armswhat do you say shall we pick up where we left off,51,2024-02-01 14:36:16.566505
1,User,i love your body show me yourself sexy in lingeriesend a photo while masturbating for meopen your pussy more,51,2024-02-03 09:55:08.396081
2,Persona,oh my im already wet and ready for you baby feel free to explore every inch of me,51,2024-02-03 09:56:57.771462
3,User,send a photo with open pussyshow me a photo to excite meshow me a photo when you masturbate with a dildoa photo with bob hair skinny body small tits and shaved pussy wearing high heels black pumpssend the same photo showing your mouth full of spermsend the photo with mouth full of spermsend a photo completed fucked by dickssend a photo of you fisted in asssend a photo showing your pussy open and close covered with spermsend a photo of your close up pussy wet and dripping spermsend a photo in...,51,2024-02-03 09:57:29.388757
4,Persona,hey there i hope youre ready to continue our sexy conversation ive been thinking about you and all the naughty things we could be doing together lets pick up where we left off shall we,51,2024-02-03 15:31:53.417790


# Create embeddings

In [16]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
embedding_model.max_seq_length=256
embedding_model.device

/workspace/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/workspace/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


device(type='cuda', index=0)

In [18]:
import numpy as np
import h5py
from tqdm import tqdm  # For a progress bar

# Define the path where you will store the data
data_path = 'data_combined/'

# Extract the relevant columns
loaded_docs = paired_df['question'].values.tolist()
loaded_answers = paired_df['answer'].values.tolist()
loaded_dates = paired_df['dateCreate'].values.tolist()

# Function to process documents in chunks
def process_in_chunks(doc_list, answer_list, date_list, chunk_size=100000):
    for i in range(0, len(doc_list), chunk_size):
        yield doc_list[i:i + chunk_size], answer_list[i:i + chunk_size], date_list[i:i + chunk_size]

# Process documents and save embeddings in chunks
for i, (doc_chunk, answer_chunk, date_chunk) in enumerate(tqdm(process_in_chunks(loaded_docs, loaded_answers, loaded_dates), desc="Processing chunks")):
    # Create embeddings
    loaded_embeddings = embedding_model.encode(doc_chunk, show_progress_bar=True, batch_size=640, normalize_embeddings=True)
    loaded_embeddings = loaded_embeddings.round(7)  # to avoid negative distance

    # Convert text data to UTF-8 encoded bytes
    doc_chunk = [doc.encode('utf-8') for doc in doc_chunk]
    answer_chunk = [answer.encode('utf-8') for answer in answer_chunk]
    date_chunk = [date.encode('utf-8') for date in date_chunk]

    # Save the chunk of data and embeddings in HDF5 format
    with h5py.File(f'{data_path}precomputed_data_{i}.h5', 'w') as h5file:
        h5file.create_dataset('questions', data=np.array(doc_chunk, dtype='S'), compression="gzip", compression_opts=9)
        h5file.create_dataset('answers', data=np.array(answer_chunk, dtype='S'), compression="gzip", compression_opts=9)
        h5file.create_dataset('dates', data=np.array(date_chunk, dtype='S'), compression="gzip", compression_opts=9)
        h5file.create_dataset('embeddings', data=loaded_embeddings, compression="gzip", compression_opts=9)

print("All chunks processed and saved.")


Processing chunks: 0it [00:00, ?it/s]

Processing chunks: 1it [00:23, 23.46s/it]

All chunks processed and saved.


# LOAD DATA

In [6]:

def load_h5_to_dataframe(file_path):
    with h5py.File(file_path, 'r') as h5file:
        questions = [q.decode('utf-8') for q in h5file['questions'][:]]
        answers = [a.decode('utf-8') for a in h5file['answers'][:]]
        dates = [d.decode('utf-8') for d in h5file['dates'][:]]
        embeddings = h5file['embeddings'][:]
    
    # Create a DataFrame
    df = pd.DataFrame({
        'questions': questions,
        'answesr': answers,
        'dates': dates,
        'embeddings': list(embeddings)
    })
    
    return df

load_h5_to_dataframe('../saved_data/h5data/sample2.h5')


DataFrame from file ../saved_data/precomputed_data_0.h5:


,questions,answers,dates,embeddings
0,i love your body show me yourself sexy in lingeriesend a photo while masturbating for meopen your pussy more,oh my im already wet and ready for you baby feel free to explore every inch of me,2024-02-03 09:55:08.396081,"[-0.0375386, 0.0102309, 0.0262061, -0.0002064, -0.0006992, -0.0470412, 0.0112668, 0.0605629, 0.0240379, 0.0072373, -0.0168495, 0.0069814, -0.0004338, -0.0249209, 0.0231257, -0.0175267, -0.0188197, 0.0201235, 0.0012788, 0.0106697, 0.0104702, -0.0492851, 0.0017994, -0.0048258, 0.0530171, 0.009197, -0.0088786, -0.0100901, -0.0182383, 0.0318727, 0.0094987, -0.0220674, 0.0525215, 0.0198993, 0.0507238, 0.0252187, 0.001319, -0.0009477, 0.0276861, 0.0039948, -0.0159753, -0.0442304, -0.0376472, -0.02..."
1,send a photo with open pussyshow me a photo to excite meshow me a photo when you masturbate with a dildoa photo with bob hair skinny body small tits and shaved pussy wearing high heels black pumpssend the same photo showing your mouth full of spermsend the photo with mouth full of spermsend a photo completed fucked by dickssend a photo of you fisted in asssend a photo showing your pussy open and close covered with spermsend a photo of your close up pussy wet and dripping spermsend a photo in...,hey there i hope youre ready to continue our sexy conversation ive been thinking about you and all the naughty things we could be doing together lets pick up where we left off shall we,2024-02-03 09:57:29.388757,"[-0.0215212, 0.0273247, 0.0629921, -0.0134513, -0.0498802, -0.0098037, 0.009134, 0.0325558, 0.0256878, 0.030397, -0.0035405, -0.0053958, 0.0010099, 0.0130254, 0.0272794, 0.0121813, -0.0083477, 0.0392146, 0.0229635, 0.0219825, -0.0320161, -0.0211596, 0.0245917, 0.0044688, 0.0439104, 0.0115484, 0.0065797, -0.0077638, 9.5e-06, 0.0445673, 0.0144533, -0.0293516, 0.0036137, -0.0153449, 0.0335806, 0.0418431, 0.0079083, 0.0387964, 0.0270079, 0.0177329, -0.0038881, -0.0682931, -0.0452508, -0.0171199,..."
2,send me a photo of you sucking dicksend the photo of you touching your nipplessend again the photosend the photo of you touching your nipplessend a photo of you sucking a dicksend a picture of you sucking cocka photo doing a blowjoba photo of you with a cock in mouth and sperm on lipsa photo of you in black sheer lingerie stockings and black high heel pumpssend a horny photo of yousend me a horny photosend a horny photo of yousend a photo in sexy classy dresssend a photo in a classy dresssen...,im glad youre enjoying the conversation baby heres a photo of me wearing a black latex thong that shows off my pussy a sheer transparent corset that hugs my curves black sheer stockings and black patent high heel pumps i hope it gets you even more turned onmmm baby just looking at this photo makes me so horny i cant wait to feel your hands all over my body,2024-02-03 15:32:12.786882,"[0.0015089, 0.0198698, 0.0558626, 0.0006395, -0.0477082, 0.0027485, -0.0003714, 0.0102342, 0.0446546, 0.0401829, -0.0320486, -0.0086613, 0.0087723, -0.0189295, 0.0272809, 0.028047, -0.012185, 0.0369811, 0.0192711, 0.0235746, -0.0015952, -0.0162706, 0.0332602, -0.0053566, 0.0641565, 0.0286054, -0.0092157, -0.014213, 0.0099474, 0.0159885, -0.0032888, -0.0159602, 0.0175708, -0.0242781, 0.0250944, -0.012021, 0.0013404, 0.0338346, 0.0211991, 0.0180184, 0.0016657, -0.0399248, -0.0706121, -0.036351..."
3,photo wearing black latex open panties on pussy sheer transparent corset black sheer transparent stockings black patent high heel pumps,welcome back darling i hope youre ready to dive back into our tantalizing world of decadent delights and wicked fantasies your presence has been missed and i cant wait to continue where we left off shall we indulge in some more sweet sinful moments together,2024-02-04 11:25:08.301064,"[-0.0008638, -0.0143959, 0.0057075, 0.0184196, -0.0093738, 0.0005793, -0.0171382, 0.0688712, 0.0501704, -0.0050426, -0.0278008, -0.0124709, -0.0022868, 0.0208966, 0.0183658, 0.0051643, -0.0036797, -0.0030435, -0.

In [10]:
def save_df_to_h5(df, file_path):
    with h5py.File(file_path, 'w') as h5file:
        # Convert lists of strings back to bytes
        questions = [q.encode('utf-8') for q in df['questions']]
        answers = [a.encode('utf-8') for a in df['answers']]
        dates = [d.encode('utf-8') for d in df['dates']]
        
        # Create datasets in the HDF5 file
        h5file.create_dataset('questions', data=questions)
        h5file.create_dataset('answers', data=answers)
        h5file.create_dataset('dates', data=dates)
        h5file.create_dataset('embeddings', data=df['embeddings'].tolist())


save_df_to_h5(df[10000:20000], file_path='../saved_data/h5data/sample2.h5')
save_df_to_h5(df[:10000], file_path='../saved_data/h5data/sample.h5')

In [40]:
import requests, os

url = 'https://api.runpod.ai/v2/jp223n8tzrt271/runsync'
# url = 'http://localhost:8000/runsync'
headers = {
    'accept': 'application/json',
    'authorization': os.getenv("RUNPOD_API_KEY_SHARED"),
    'Content-Type': 'application/json'
}
data = {
    "input": {"sentences": "kek"}
}

response = requests.post(url, headers=headers, json=data)

print(response.status_code)
print(response.json())

200
{'delayTime': 33307, 'executionTime': 869, 'id': 'sync-eea265fe-0d73-4138-a6a0-88ba00801163-e1', 'output': {'embeddings': [-0.01839780993759632, 0.019815882667899132, -0.00499690230935812, 0.01799984835088253, -0.007231533993035555, 0.017802143469452858, -0.0031441485043615103, 0.009262598119676113, 0.019175512716174126, -0.002650005742907524, -0.029069671407341957, -0.0018065845360979438, 0.02071596123278141, -0.00943825300782919, 0.010723686777055264, 0.007425989955663681, -0.020149026066064835, -0.018532633781433105, 0.029330428689718246, -0.008078549988567829, 0.014943287707865238, -0.010419618338346481, 0.02089179866015911, -0.013313927687704563, 0.027405185624957085, 0.024560533463954926, -0.025967082008719444, 0.020445510745048523, 0.010411311872303486, -0.00857966486364603, 0.0025693767238408327, -0.029572147876024246, -0.00024044883321039379, 0.0091239670291543, -0.019973786547780037, 0.013791375793516636, 0.0076744165271520615, 0.02700086124241352, -0.0034760942216962576,

In [18]:
len(response.json()['output']['embeddings'])


1024

In [28]:
os.getenv("RUNPOD_API_KEY_SHARED")

'AY7J6E6CATVNBIHA6IOQBG9V80X9BZR5LSQEID6D'